In [1]:
import sys
import logging
import pandas as pd
import os
from dotenv import load_dotenv

sys.path.append('..')
import nature_go_client
from backend.nature_go.observation.generation.summary_generation import generate_summaries

logger = logging.getLogger(__name__)



In [2]:
load_dotenv()
NATURE_GO_USERNAME = os.getenv("NATURE_GO_USERNAME")
NATURE_GO_PASSWORD = os.getenv("NATURE_GO_PASSWORD")
client = nature_go_client.NatureGoClient(username=NATURE_GO_USERNAME, password=NATURE_GO_PASSWORD)
client.login()

## Retrieve species with missing text content through Nature go API

In [3]:
species_list = pd.DataFrame(client.get_labeled_species(limit=5, multiplechoicequestions=False))
species_list = species_list.sort_values(by='num_observations_total', ascending=False)
species_list

http://nature-go.edouardleurent.com/api/species/labeled/?limit=5&offset=0&descriptions=False&multiplechoicequestions=False


,id,type,scientificNameWithoutAuthor,commonNames,genus,family,gbif_id,powo_id,wikipedia_word_count,number_of_occurrences,...,illustration,illustration_transparent,illustration_reference,illustration_reference_transparent,reference_image_url,rarity,illustration_url,display_name,num_observations_total,num_questions_total
0,37953,bird,Parus major,[Great Tit],Parus,Paridae,,,4415,NaN,...,http://nature-go.edouardleurent.com/media/spec...,http://nature-go.edouardleurent.com/media/spec...,None,None,,Very Common,http://nature-go.edouardleurent.com/media/spec...,Great Tit,2,0
1,38215,bird,Chloris chloris,[European Greenfinch],Chloris,Fringillidae,,,991,NaN,...,http://nature-go.edouardleurent.com/media/spec...,http://nature-go.edouardleurent.com/media/spec...,None,None,,Very Common,http://nature-go.edouardleurent.com/media/spec...,European Greenfinch,2,0
2,182,plant,Euphorbia amygdaloides,"[Wood Spurge, Laptele câinelui (RO), Greater H...",Euphorbia,Euphorbiaceae,3065239.0,345568-1,155,2155.0,...,http://nature-go.edouardleurent.com/media/spec...,http://nature-go.edouardleurent.com/media/spec...,None,None,,Common,http://nature-go.edouardleurent.com/media/spec...,Wood Spurge,1,0
3,3120,plant,Narcissus bicolor,"[Two-color daffodil, Two-coloured Daffodil]",Narcissus,Amaryllidaceae,2858359.0,65928-1,-1,28.0,...,None,None,None,None,,Rare,http://nature-go.edouardleurent.com/static/img...,Two-color daffodil,1,0
4,4681,plant,Papaver cambricum,"[Welsh Poppy, Poppy]",Papaver,Papaveraceae,3602683.0,673433-1,495,10.0,...,None,None,None,None,,Rare,http://nature-go.edouardleurent.com/static/img...,Welsh Poppy,1,0


In [4]:
species = species_list.iloc[0]
print(species)

id                                                                                37953
type                                                                               bird
scientificNameWithoutAuthor                                                 Parus major
commonNames                                                                 [Great Tit]
genus                                                                             Parus
family                                                                          Paridae
gbif_id                                                                                
powo_id                                                                                
wikipedia_word_count                                                               4415
number_of_occurrences                                                               NaN
occurences_cdf                                                                      NaN
rarity_gpt                      

## Run generation

In [ ]:
import backend.nature_go.observation.generation.gemini as gemini
gemini.configure(os.getenv('GOOGLE_API_KEY'))

In [9]:
import backend.nature_go.observation.generation.summary_generation as summary_generation
import prompts.summary_prompt

summary = summary_generation.generate_summaries(
    generate_text=backend.nature_go.observation.generation.gemini.generate_text,
    species=species,
    prompt=prompts.summary_prompt.summary_v7
)

Today I’ve been observing the Great Tit, a rather common bird here in Europe. A small fellow, but with a strikingly bright yellow chest and bold black markings, almost as if he’s wearing a tiny tuxedo. I’ve been particularly amused by the stories I’ve heard about these birds – they’ve apparently learned to break into milk bottles! Imagine, a little bird cracking open a bottle just to get a taste of the cream at the top. Now, that’s a clever little fellow! 

<BREAK_1>
The Great Tit is not easily fooled, however. It’s been described as one of the most intelligent bird species, having a complex system of calls and songs – perhaps even an elaborate language of their own! These birds are quite the conversationalists, their “teacher, teacher” calls echoing through the woods. 

<BREAK_2>
This tit is a master of survival,  making the most of available resources and even resorting to eating the brains of hibernating bats in harsh winters. I had never imagined a songbird being a predator, but th

In [10]:
print('############ Full ############')
print(summary['original'])
print('\n############ Part 1 ############')
print(summary['part_1'])
print('\n############ Part 2 ############')
print(summary['part_2'])
print('\n############ Part 3 ############')
print(summary['part_3'])

############ Full ############
Today I’ve been observing the Great Tit, a rather common bird here in Europe. A small fellow, but with a strikingly bright yellow chest and bold black markings, almost as if he’s wearing a tiny tuxedo. I’ve been particularly amused by the stories I’ve heard about these birds – they’ve apparently learned to break into milk bottles! Imagine, a little bird cracking open a bottle just to get a taste of the cream at the top. Now, that’s a clever little fellow! 

<BREAK_1>
The Great Tit is not easily fooled, however. It’s been described as one of the most intelligent bird species, having a complex system of calls and songs – perhaps even an elaborate language of their own! These birds are quite the conversationalists, their “teacher, teacher” calls echoing through the woods. 

<BREAK_2>
This tit is a master of survival,  making the most of available resources and even resorting to eating the brains of hibernating bats in harsh winters. I had never imagined a so

In [11]:
species

id                                                                                37953
type                                                                               bird
scientificNameWithoutAuthor                                                 Parus major
commonNames                                                                 [Great Tit]
genus                                                                             Parus
family                                                                          Paridae
gbif_id                                                                                
powo_id                                                                                
wikipedia_word_count                                                               4415
number_of_occurrences                                                               NaN
occurences_cdf                                                                      NaN
rarity_gpt                      

In [12]:
import backend.nature_go.observation.generation.question_generation as question_generation
import importlib
importlib.reload(question_generation)

material = ' '.join([summary['part_1'], summary['part_2'], summary['part_3']])
questions, _ = question_generation.generate_questions(
    generate_text=backend.nature_go.observation.generation.gemini.generate_text,
    species=species,
    material=material,
)
questions

[{'question': 'What is a distinctive physical feature of the Great Tit?',
  'choices': ['A bright yellow chest and black markings',
   'A long, pointed beak',
   'A bright red crest',
   'A long, flowing tail'],
  'correct_choice': 0},
 {'question': 'What is one of the unusual behaviors that Great Tits are known for?',
  'choices': ['Building intricate nests in tree hollows',
   'Migrating long distances in winter',
   'Learning to open milk bottles',
   'Singing complex melodies'],
  'correct_choice': 2},
 {'question': "What does the passage suggest about the Great Tit's intelligence?",
  'choices': ['It is a very simple bird with limited cognitive abilities',
   'It is one of the most intelligent bird species',
   'It is only intelligent when it comes to finding food',
   'It is not very intelligent, but it is resourceful'],
  'correct_choice': 1}]